# State vs Legacy Memory Optimization

Compare grid-search results for $(\gamma, w_0)$ between the state-based implementation (`src/Bayesian_state`) and the legacy Bayesian pipeline (`src/Bayesian`) on a shared subject.

In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / "src").is_dir():
    PROJECT_ROOT = PROJECT_ROOT.parent

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

DATA_PATH = PROJECT_ROOT / "data" / "processed" / "Task2_processed.csv"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"

print(f"Project root: {PROJECT_ROOT}")
print(f"Data path: {DATA_PATH}")

Project root: D:\Research\nips_reviewing\CategoryLearning\StateBased\CategoryLearning
Data path: D:\Research\nips_reviewing\CategoryLearning\StateBased\CategoryLearning\data\processed\Task2_processed.csv


In [2]:
import numpy as np
import pandas as pd

from copy import deepcopy
from src.Bayesian_state.utils.load_config import MODEL_STRUCT
from src.Bayesian_state.utils.state_grid_optimizer import StateModelGridOptimizer
from src.Bayesian.utils.optimizer import Optimizer
from src.Bayesian.problems.config import config_fgt
from src.Bayesian.problems.modules.memory import BaseMemory

INFO:cat-learning:logger is running normally.


D:\Research\nips_reviewing\CategoryLearning\StateBased\CategoryLearning\logs\Run_20251105_235006.log
{'base_model': {'modules': {'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule', 'kwargs': {}}}, 'agenda': ['likelihood_mod', '__self__']}, 'default_model': {'modules': {}, 'agenda': ['__self__']}, 'm_model': {'modules': {'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule', 'kwargs': {'beta': 5.0}}, 'memory_mod': {'class': 'src.Bayesian_state.problems.modules.memory.DualMemoryModule', 'kwargs': {'w0': 0.8, 'gamma': 0.8}}}, 'agenda': ['likelihood_mod', 'memory_mod']}, 'pmh_model': {'modules': {'perception_mod': {'class': 'src.Bayesian_state.problems.modules.perception.PerceptionModule'}, 'hypo_transitions_mod': {'class': 'src.Bayesian_state.problems.modules.hypo_transitions.FixedNumHypothesisModule', 'kwargs': {'fixed_hypo_num': 20, 'init_strategy': 'random', 'transition_mode': 'top_posterior', 'throw_num

In [3]:
subject_id = 17
gamma_resolution = 5
gamma_grid = np.linspace(0.4, 0.9, gamma_resolution)
w0_upper = 0.15
w0_grid = np.array([w0_upper / (idx + 1) for idx in range(gamma_resolution)], dtype=float)

legacy_module_config = {
    subject_id: {
        "memory": (
            BaseMemory,
            {
                "personal_memory_range": {
                    "gamma": (float(gamma_grid.min()), float(gamma_grid.max())),
                    "w0": (float(w0_grid.min()), float(w0_grid.max())),
                },
                "param_resolution": len(gamma_grid),
            },
        ),
    }
}

legacy_optimizer = Optimizer(legacy_module_config, n_jobs=1)
legacy_optimizer.prepare_data(DATA_PATH)
legacy_results = legacy_optimizer.optimize_params_with_subs_parallel(
    config_fgt,
    subjects=[subject_id],
    window_size=16,
    grid_repeat=1,
    mc_samples=1,
    stop_at=0.5,
 )

legacy_best_params = legacy_results[subject_id]["best_params"]
legacy_best_gamma = float(legacy_best_params["gamma"])
legacy_best_w0 = float(legacy_best_params["w0"])
legacy_best_sequence = legacy_results[subject_id]["best_step_results"]
legacy_best_beta = float(legacy_best_sequence[-1]["best_beta"])

print(f"Legacy best gamma: {legacy_best_gamma:.4f}")
print(f"Legacy best w0: {legacy_best_w0:.4f}")
print(f"Legacy best beta (likelihood): {legacy_best_beta:.4f}")
print(f"Legacy mean error: {legacy_results[subject_id]['best_error']:.6f}")

Using parallel optimization for multiple subjects.


Processing tasks: 100%|██████████████████████████████████████████| 25/25 [1:02:15<00:00, 149.40s/it]
INFO:src.Bayesian.utils.optimizer:Subject 17 has enough grid repeat samples.
Processing tasks: 100%|██████████████████████████████████████████| 25/25 [1:02:15<00:00, 149.40s/it]
INFO:src.Bayesian.utils.optimizer:Subject 17 has enough grid repeat samples.


Legacy best gamma: 0.4000
Legacy best w0: 0.0300
Legacy best beta (likelihood): 25.3601
Legacy mean error: 0.114325


In [4]:
state_engine_config = deepcopy(MODEL_STRUCT["pm_model"])
state_engine_config["modules"]["likelihood_mod"]["kwargs"]["beta"] = legacy_best_beta

state_optimizer = StateModelGridOptimizer(
    state_engine_config,
    processed_data_dir=PROCESSED_DIR,
 )
state_optimizer.prepare_data(DATA_PATH)
state_results = state_optimizer.grid_search_subject(
    subject_id=subject_id,
    gamma_grid=gamma_grid,
    w0_grid=w0_grid,
    window_size=16,
    stop_at=0.5,
    max_trials=120,
 )
state_best = state_results["best"]

print(f"StateModel best gamma: {state_best.gamma:.4f}")
print(f"StateModel best w0: {state_best.w0:.4f}")
print(f"StateModel mean error: {state_best.mean_error:.6f}")

INFO:cat-learning:  - Module 'likelihood_mod' registered as 'self.likelihood_mod'.
INFO:cat-learning:  - Module 'memory_mod' registered as 'self.memory_mod'.
INFO:cat-learning:  - Module 'perception_mod' registered as 'self.perception_mod'.
INFO:cat-learning:  - Module 'memory_mod' registered as 'self.memory_mod'.
INFO:cat-learning:  - Module 'perception_mod' registered as 'self.perception_mod'.


{'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule', 'kwargs': {'beta': 25.360095973987043}}, 'memory_mod': {'class': 'src.Bayesian_state.problems.modules.memory.DualMemoryModule', 'kwargs': {'w0': 0.15, 'gamma': 0.4}}, 'perception_mod': {'class': 'src.Bayesian_state.problems.modules.perception.PerceptionModule', 'kwargs': {'mean': [0.011870610017748164, 0.0013730145163006638, 0.008240268070487557, -0.00022768464291453663], 'std': [0.04299526437253106, 0.03963304694303766, 0.041823486395900035, 0.019587025539177582], 'subject_id': 17}}}
name: likelihood_mod mod_kwargs: {'beta': 25.360095973987043}
name: memory_mod mod_kwargs: {'w0': 0.15, 'gamma': 0.4}
name: perception_mod mod_kwargs: {'mean': [0.011870610017748164, 0.0013730145163006638, 0.008240268070487557, -0.00022768464291453663], 'std': [0.04299526437253106, 0.03963304694303766, 0.041823486395900035, 0.019587025539177582], 'subject_id': 17}


INFO:cat-learning:  - Module 'likelihood_mod' registered as 'self.likelihood_mod'.
INFO:cat-learning:  - Module 'memory_mod' registered as 'self.memory_mod'.
INFO:cat-learning:  - Module 'perception_mod' registered as 'self.perception_mod'.
INFO:cat-learning:  - Module 'memory_mod' registered as 'self.memory_mod'.
INFO:cat-learning:  - Module 'perception_mod' registered as 'self.perception_mod'.


{'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule', 'kwargs': {'beta': 25.360095973987043}}, 'memory_mod': {'class': 'src.Bayesian_state.problems.modules.memory.DualMemoryModule', 'kwargs': {'w0': 0.075, 'gamma': 0.4}}, 'perception_mod': {'class': 'src.Bayesian_state.problems.modules.perception.PerceptionModule', 'kwargs': {'mean': [0.011870610017748164, 0.0013730145163006638, 0.008240268070487557, -0.00022768464291453663], 'std': [0.04299526437253106, 0.03963304694303766, 0.041823486395900035, 0.019587025539177582], 'subject_id': 17}}}
name: likelihood_mod mod_kwargs: {'beta': 25.360095973987043}
name: memory_mod mod_kwargs: {'w0': 0.075, 'gamma': 0.4}
name: perception_mod mod_kwargs: {'mean': [0.011870610017748164, 0.0013730145163006638, 0.008240268070487557, -0.00022768464291453663], 'std': [0.04299526437253106, 0.03963304694303766, 0.041823486395900035, 0.019587025539177582], 'subject_id': 17}


INFO:cat-learning:  - Module 'likelihood_mod' registered as 'self.likelihood_mod'.
INFO:cat-learning:  - Module 'memory_mod' registered as 'self.memory_mod'.
INFO:cat-learning:  - Module 'perception_mod' registered as 'self.perception_mod'.
INFO:cat-learning:  - Module 'memory_mod' registered as 'self.memory_mod'.
INFO:cat-learning:  - Module 'perception_mod' registered as 'self.perception_mod'.


...

{'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule', 'kwargs': {'beta': 25.360095973987043}}, 'memory_mod': {'class': 'src.Bayesian_state.problems.modules.memory.DualMemoryModule', 'kwargs': {'w0': 0.049999999999999996, 'gamma': 0.9}}, 'perception_mod': {'class': 'src.Bayesian_state.problems.modules.perception.PerceptionModule', 'kwargs': {'mean': [0.011870610017748164, 0.0013730145163006638, 0.008240268070487557, -0.00022768464291453663], 'std': [0.04299526437253106, 0.03963304694303766, 0.041823486395900035, 0.019587025539177582], 'subject_id': 17}}}
name: likelihood_mod mod_kwargs: {'beta': 25.360095973987043}
name: memory_mod mod_kwargs: {'w0': 0.049999999999999996, 'gamma': 0.9}
name: perception_mod mod_kwargs: {'mean': [0.011870610017748164, 0.0013730145163006638, 0.008240268070487557, -0.00022768464291453663], 'std': [0.04299526437253106, 0.03963304694303766, 0.041823486395900035, 0.019587025539177582], 'subject_id': 17}


INFO:cat-learning:  - Module 'likelihood_mod' registered as 'self.likelihood_mod'.
INFO:cat-learning:  - Module 'memory_mod' registered as 'self.memory_mod'.
INFO:cat-learning:  - Module 'perception_mod' registered as 'self.perception_mod'.
INFO:cat-learning:  - Module 'memory_mod' registered as 'self.memory_mod'.
INFO:cat-learning:  - Module 'perception_mod' registered as 'self.perception_mod'.


{'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule', 'kwargs': {'beta': 25.360095973987043}}, 'memory_mod': {'class': 'src.Bayesian_state.problems.modules.memory.DualMemoryModule', 'kwargs': {'w0': 0.0375, 'gamma': 0.9}}, 'perception_mod': {'class': 'src.Bayesian_state.problems.modules.perception.PerceptionModule', 'kwargs': {'mean': [0.011870610017748164, 0.0013730145163006638, 0.008240268070487557, -0.00022768464291453663], 'std': [0.04299526437253106, 0.03963304694303766, 0.041823486395900035, 0.019587025539177582], 'subject_id': 17}}}
name: likelihood_mod mod_kwargs: {'beta': 25.360095973987043}
name: memory_mod mod_kwargs: {'w0': 0.0375, 'gamma': 0.9}
name: perception_mod mod_kwargs: {'mean': [0.011870610017748164, 0.0013730145163006638, 0.008240268070487557, -0.00022768464291453663], 'std': [0.04299526437253106, 0.03963304694303766, 0.041823486395900035, 0.019587025539177582], 'subject_id': 17}


INFO:cat-learning:  - Module 'likelihood_mod' registered as 'self.likelihood_mod'.
INFO:cat-learning:  - Module 'memory_mod' registered as 'self.memory_mod'.
INFO:cat-learning:  - Module 'perception_mod' registered as 'self.perception_mod'.
INFO:cat-learning:  - Module 'memory_mod' registered as 'self.memory_mod'.
INFO:cat-learning:  - Module 'perception_mod' registered as 'self.perception_mod'.


{'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule', 'kwargs': {'beta': 25.360095973987043}}, 'memory_mod': {'class': 'src.Bayesian_state.problems.modules.memory.DualMemoryModule', 'kwargs': {'w0': 0.03, 'gamma': 0.9}}, 'perception_mod': {'class': 'src.Bayesian_state.problems.modules.perception.PerceptionModule', 'kwargs': {'mean': [0.011870610017748164, 0.0013730145163006638, 0.008240268070487557, -0.00022768464291453663], 'std': [0.04299526437253106, 0.03963304694303766, 0.041823486395900035, 0.019587025539177582], 'subject_id': 17}}}
name: likelihood_mod mod_kwargs: {'beta': 25.360095973987043}
name: memory_mod mod_kwargs: {'w0': 0.03, 'gamma': 0.9}
name: perception_mod mod_kwargs: {'mean': [0.011870610017748164, 0.0013730145163006638, 0.008240268070487557, -0.00022768464291453663], 'std': [0.04299526437253106, 0.03963304694303766, 0.041823486395900035, 0.019587025539177582], 'subject_id': 17}
StateModel best gamma: 0.6500
StateModel best w

In [5]:
comparison_df = pd.DataFrame([
    {
        "pipeline": "legacy",
        "gamma": legacy_best_gamma,
        "w0": legacy_best_w0,
        "beta": legacy_best_beta,
        "mean_error": legacy_results[subject_id]["best_error"],
    },
    {
        "pipeline": "state",
        "gamma": state_best.gamma,
        "w0": state_best.w0,
        "beta": legacy_best_beta,
        "mean_error": state_best.mean_error,
    },
])
comparison_df

,pipeline,gamma,w0,beta,mean_error
0,legacy,0.40,0.0300,25.360096,0.114325
1,state,0.65,0.0375,25.360096,0.094555


In [6]:
grid_df = pd.DataFrame(
    [
        {"gamma": item.gamma, "w0": item.w0, "mean_error": item.mean_error}
        for item in state_results["grid"]
    ]
).sort_values("mean_error")
grid_df.head()

,gamma,w0,mean_error
13,0.650,0.0375,0.094555
14,0.650,0.0300,0.094925
12,0.650,0.0500,0.095074
15,0.775,0.1500,0.095378
2,0.400,0.0500,0.095818


In [7]:
from src.Bayesian_state.problems import StateModel

subject_frame = state_optimizer._get_subject_frame(subject_id, stop_at=0.5)
stimulus, choices, feedback, categories = state_optimizer._extract_arrays(subject_frame, max_trials=120)

legacy_combo = state_optimizer._evaluate_combination(
    subject_id,
    int(legacy_results[subject_id]["condition"]),
    (stimulus, choices, feedback, categories),
    legacy_best_gamma,
    legacy_best_w0,
    16,
 )

print(f"StateModel error with legacy params: {legacy_combo.mean_error:.6f}")

INFO:cat-learning:  - Module 'likelihood_mod' registered as 'self.likelihood_mod'.
INFO:cat-learning:  - Module 'memory_mod' registered as 'self.memory_mod'.
INFO:cat-learning:  - Module 'perception_mod' registered as 'self.perception_mod'.
INFO:cat-learning:  - Module 'memory_mod' registered as 'self.memory_mod'.
INFO:cat-learning:  - Module 'perception_mod' registered as 'self.perception_mod'.


{'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule', 'kwargs': {'beta': 25.360095973987043}}, 'memory_mod': {'class': 'src.Bayesian_state.problems.modules.memory.DualMemoryModule', 'kwargs': {'w0': 0.03, 'gamma': 0.4}}, 'perception_mod': {'class': 'src.Bayesian_state.problems.modules.perception.PerceptionModule', 'kwargs': {'mean': [0.011870610017748164, 0.0013730145163006638, 0.008240268070487557, -0.00022768464291453663], 'std': [0.04299526437253106, 0.03963304694303766, 0.041823486395900035, 0.019587025539177582], 'subject_id': 17}}}
name: likelihood_mod mod_kwargs: {'beta': 25.360095973987043}
name: memory_mod mod_kwargs: {'w0': 0.03, 'gamma': 0.4}
name: perception_mod mod_kwargs: {'mean': [0.011870610017748164, 0.0013730145163006638, 0.008240268070487557, -0.00022768464291453663], 'std': [0.04299526437253106, 0.03963304694303766, 0.041823486395900035, 0.019587025539177582], 'subject_id': 17}
StateModel error with legacy params: 0.102149
S